# Photo Conductivity

When the electronic state of the defect interacts with radiation, carriers can be excited to the conduction band. This is called photoconductivity. To help elucidate how likely a defect is to cause photoconductivity, we can look at the band-decomposed contribution to the frequency-dependent dielectric function.
Here we have replicated `VASP`'s density-density optical response calculated in the independent-particle picture in the `pymatgen.io.vasp.optics` module, and a allowed users to control which dipole matrix elements are accounted for by masking the dipole matrix elements stored in the `WAVEDER` file and manipulate the Fermi level to populate or depopulate the defect state.
This essentially allows us to calculate the optical response for (defect)→(conduction band) and (valence band)→(defect) transitions in isolation.

For more information on how `VASP` handles optical response, see the [VASP documentation](https://www.vasp.at/wiki/index.php/LOPTICS).

For more details on how the python code works, please see the documentation for the `pymatgen.io.vasp.optics` module.


In [ ]:
# disable tqdm progress bar
from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)
# disable warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
from pathlib import Path
from pymatgen.analysis.defects.ccd import HarmonicDefect, get_SRH_coefficient
from pymatgen.io.vasp.optics import DielectricFunctionCalculator, Spin, Vasprun, Waveder
import numpy as np
from matplotlib import pyplot as plt
import bisect
import collections
TEST_FILES = Path("../../../tests/test_files/v_Ga/")

To compute the optical response of a defect we can use the `HarmonicDefect` object again.
Although this time, since we don't need to consider the vibrational modes of the defect, we only need to pass in the directory containing the relaxed structure and the `WAVEDER` file.
Since the `PROCAR` file is also present in the directory, the `(band, kpoint, spin)` indices representing the singular, most localized, state will be automatically determined.

In [ ]:
dir0 = TEST_FILES / "ccd_0_-1" / "optics"
hd0 = HarmonicDefect.from_directories(directories=[dir0])
print(f"The defect band is {hd0.defect_band}")
print(f"The vibrational frequency is omega={hd0.omega} in this case is gibberish.")

## Identification of the Defect Band

We can first check that the `defect_band`, identified using the `PROCAR` file, is indeed an isolated flat band in the middle of the band gap.
While this is not always the case since we are looking at Kohn-Sham states, this is usually a good starting point.
The user can refine the definition of the defect band by passing in a list of `(band, kpoint, spin)` indices to the `defect_band` argument of the `HarmonicDefect` object.

In [ ]:

def print_eigs(vr, bwind=5, defect_bands = ()):
    """Print the eigenvalues in a small band window around the Fermi level.

    Args:
        vr (Vasprun): The Vasprun object.
        bwind (int): The number of bands above and below the Fermi level to print.
        defect_bands (list): A list of tuples of the form (band index, kpt index, spin index)
    """
    def _get_spin_idx(spin):
        if spin == Spin.up:
            return 0
        return 1
    occ = vr.eigenvalues[Spin.up][0, :, 1]
    fermi_idx = bisect.bisect_left(occ, -0.5, key=lambda x: -x) 
    output = collections.defaultdict(list)
    for k, spin_eigs in vr.eigenvalues.items():
        spin_idx = _get_spin_idx(k)
        for kpt in range(spin_eigs.shape[0]):
            for ib in range(fermi_idx - bwind, fermi_idx + bwind):
                e, o = spin_eigs[kpt, ib, :]
                idx = (ib, kpt, spin_idx)
                if idx in defect_bands:
                    e_out = f"{e:7.4f}*"
                else:
                    e_out = f"{e:8.5f}"
                output[(ib)].append(e_out)
    print("band s=0,k=0  s=0,k=1  s=1,k=0  s=1,k=1")
    for ib, eigs in output.items():
        print(f"{ib:3d} {' '.join(eigs)}")

print_eigs(vr=hd0.vrun, defect_bands=hd0.defect_band)

From the eigenvalues above, we can see that isolated defect band is `[(138, 0, 1), (138, 1, 1)]` which matches with starred values determined by the data from inverse participation ratio (IPR) analysis.

## Frequency-Dependent Dielectric Function

The frequency-dependent dielectric function is calculated using the `get_dielectric_function` method of the `HarmonicDefect` object.
Which requires the `waveder` attribute of the `HarmonicDefect` object to be set.

In [ ]:
hd0.waveder = Waveder.from_binary(dir0 / "WAVEDER")

The (x,x) component of the complex dielectric function tensor for the (VBM)→(defect) and (defect)→(CBM) transitions can be obtained by calling the `get_dielectric_function` method with `idir=0, jdir=0`.

In [ ]:
energy, eps_vbm, eps_cbm = hd0.get_dielectric_function(idir=0,jdir=0)
# plotting
plt.plot(energy, np.imag(eps_vbm), label="VBM")
plt.plot(energy, np.imag(eps_cbm), label="CBM")
plt.xlabel("Photon Energy (eV)")
plt.ylabel(r"$\varepsilon_{\rm imag}$")
plt.xlim(0, 8)
plt.legend();

From the dielectric function above we see that this particular defect (the Ga vacancy in the neutral `q=0` state) is much better at capturing electrons from the valence band via dipole transitions than it is at emitting electrons to the conduction band via dipole transitions.

Of course for a complete picture of photoconductivity, the Frank-Condon type ofr vibrational state transition should also be considered, but we are already pushing the limits of what is acceptable in the independent-particle picture so we will leave that for another time.
